<h2><center><b><i>Cluster bomb</b></i>: Uncovering Patterns in Terrorist Group Beliefs and Attacks</center></h2>

#### **COM-480: Data Visualization**

**Team**: Alexander Sternfeld, Silvia Romanato & Antoine Bonnet

**Dataset**: [Global Terrorism Database (GTD)](https://www.start.umd.edu/gtd/) 

**Additional dataset**: [Profiles of Perpetrators of Terrorism in the United States (PPTUS)](https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl%3A1902.1/17702)

## **Maps**
 
The goal of this notebook is to create the data underlying a world map of terrorist attacks. This data will then be used to generate an interactive map using [Kepler](https://kepler.gl), an open-source geospatial analysis tool for large-scale data sets. 

<center><p float="left">
  <img src="https://d1a3f4spazzrp4.cloudfront.net/kepler.gl/website/showcase/points-s.png" width="400" />
  <img src="https://d1a3f4spazzrp4.cloudfront.net/kepler.gl/website/showcase/lines-s.png" width="400" /> 
  <img src="https://d1a3f4spazzrp4.cloudfront.net/kepler.gl/website/showcase/hexagons-s.png" width="400" />
  <img src="https://d1a3f4spazzrp4.cloudfront.net/kepler.gl/website/showcase/heatmap-s.png" width="400" />
</p></center>


**Map 1**: A first map will display all locations of terrorist attacks from 1970 to today shown as dots on the world map. A time slider will be used to show the progression of terrorist attacks through time. 

**Map 2**: A second map will display arrows going from the base of operations of terrorist organizations to the location of terrorist attacks perpetrated by members of those organizations. Arrows will be colored depending on the ideology of each organization. 

## **Data pre-processing**

### **Map 1**: Terrorist attacks 

From the GTD, we first create an `attacks.csv` file that displays each attack's location, date, number of casualties, attack type, target type and terrorist group. This file will be used as the basis for our first map. 

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from load_data import *

pd.set_option('display.max_columns', None)

GTD = load_GTD()
PPTUS, _ = load_PPTUS()

MAP_DIR = os.path.join(DATA_DIR, 'maps')

GTD pickle file found, loading...
PPTUS pickle files found, loading...


In [2]:
# Create a datetime column by combining columns iyear, imonth, iday renaming them
GTD.rename(columns={'iyear': 'year', 'imonth': 'month', 'iday': 'day'}, inplace=True)
GTD['day'].replace(0, 1, inplace=True) # Some attacks only have year data, so we set them to January 1st 
GTD['month'].replace(0, 1, inplace=True)
GTD['datetime'] = pd.to_datetime(GTD[['year', 'month', 'day']], errors='coerce') + pd.Timedelta(hours=12)

# Keep only columns: datetime, country_txt, region_txt, latitude, longitude, nkill, attacktype1_txt, gname
GTD = GTD[['datetime', 'city', 'country_txt', 'region_txt', 'latitude', 'longitude', 'nkill', 'attacktype1_txt', 'gname', 'targtype1_txt', 'weaptype1_txt']]

# Drop rows with missing latitude, longitude
GTD = GTD.dropna(subset=['latitude', 'longitude'])

# Replace all Unknown values with nan
GTD.replace('Unknown', np.nan, inplace=True)

# Rename columns for display
GTD = GTD.rename(columns={
    'country_txt': 'Country', 
    'region_txt': 'Region', 
    'attacktype1_txt': 'Attack type',
    'nkill': 'Fatalities',
    'gname': 'Terrorist group', 
    'latitude': 'Latitude', 
    'longitude': 'Longitude', 
    'targtype1_txt': 'Target type', 
    'city': 'City', 
    'weaptype1_txt': 'Weapon used'})

# Store file in csv format 
GTD.to_csv(os.path.join(MAP_DIR, 'attacks.csv'), index=False)
GTD

,datetime,City,Country,Region,Latitude,Longitude,Fatalities,Attack type,Terrorist group,Target type,Weapon used
0,1970-07-02 12:00:00,Santo Domingo,Dominican Republic,Central America & Caribbean,18.456792,-69.951164,1.0,Assassination,MANO-D,Private Citizens & Property,NaN
1,1970-01-01 12:00:00,Mexico city,Mexico,North America,19.371887,-99.086624,0.0,Hostage Taking (Kidnapping),23rd of September Communist League,Government (Diplomatic),NaN
2,1970-01-01 12:00:00,NaN,Philippines,Southeast Asia,15.478598,120.599741,1.0,Assassination,NaN,Journalists & Media,NaN
3,1970-01-01 12:00:00,Athens,Greece,Western Europe,37.997490,23.762728,NaN,Bombing/Explosion,NaN,Government (Diplomatic),Explosives
4,1970-01-01 12:00:00,Fukouka,Japan,East Asia,33.580412,130.396361,NaN,Facility/Infrastructure Attack,NaN,Government (Diplomatic),Incendiary
...,...,...,...,...,...,...,...,...,...,...,...
4955,2021-06-30 12:00:00,Jangal Bagh,Afghanistan,South Asia,35.054772,69.196838,0.0,Bombing/Explosion,NaN,Utilities,Explosives
4956,2021-06-30 12:00:00,Muse,Myanmar,Southeast Asia,23.986739,97.897143,1.0,Hostage Taking (Kidnapping),NaN,Educational Institution,NaN
4957,2021-06-30 12:00:00,NaN,Nigeria,Sub-Saharan Africa,11.572869,13.014035,0.0,Hostage Taking (Kidnapping),Boko Haram,Religious Figures/Institutions,Firearms
4958,2021-06-30 12:00:00,NaN,Colombia,South America,3.104189,-76.333069,0.0,Bombing/Explosion,Revolutionary Armed Forces of Colombia (FARC) ...,NaN,Explosives


### **Map 2**: Group to targets arcs

This second map will focus on terrorist groups by drawing arcs from the group's country of origin to the locations of their terrorist attacks. We will then be able to see the geographical distribution of terrorist attacks for each group. 

#### Manual labelling 

To do this, we manually labelled the origin country of the top 200 groups with largest amount of fatalities in the file `top_groups.csv`. During the annotation, here were a few of the decisions that were made concerning specific groups: 

- **Allied Democratic Forces (ADF)** originated in Uganda but expanded to DRC (labelled Uganda)

- **Sikh Extremists** (i.e. Khalistan movement) originates in Punjab region = Pakistan + India (labelled Pakistan)

- **Tamils** population is mainly distributed in India, but also in Sri Lanka (labelled India)

- **Kashmir** region crosses Pakistan, Indian and China (labelled India)

- **Basque Fatherland and Freedom (ETA)** is Basque which is principally Spanish but also French (labelled Spain)

- **Movement for Oneness and Jihad in West Africa** was active in both Algeria and Mali (labelled Algeria)

- **Naga** people originate from a region split between India and Myanmar (labelled India)

- The following groups were not affiliated to a particular location and so are removed from the maps: Muslim extremists, Islamist extremists, Separatists, Death Squad, Tribesmen, Muslim Militants, Gunmen, Narco-Terrorists, Rebels, Muslim Rebels, Muslims, Jihadi-inspired extremists, Protestant extremists, Right-Wing Death Squad, Communists, Muslim Separatists, Shia Muslim extremists, Sunni Muslim extremists, Mouhadine, Rebel Military Unit, Dissident Military Mmbrs of Northern Tribal Group, Moslem Activists, Anti-Government Guerrillas, Anti-Communist Vigilante Group, Muslim Fundamentalists, White supremacists/nationalists

In [3]:
# Find all group names in GTD sorted in decreasing number of total non-NA nkill
GTD_bis = load_GTD()
GTD_bis['gname'].replace('Unknown', np.nan).dropna()
top_groups = GTD_bis[GTD_bis['gname'] != 'Unknown'].groupby('gname')['nkill'].sum(numeric_only=True).dropna().sort_values(ascending=False).reset_index()[:232]

# Manual annotation of top 100 groups country of origin
manual_locations = ['Afghanistan', 'Iraq', 'Nigeria', 'Somalia', 'Peru',
                    'Sri Lanka', 'El Salvador', 'Yemen', 'Guatemala', 
                    'Pakistan', 'Nigeria', 'Colombia', 'Turkey', 'Philippines',
                    'Iraq', 'Rwanda', 'Afghanistan', 'Afghanistan', 'Yemen', 
                    'Uganda', 'Syria', 'Uganda', 'Pakistan', 'Angola', 'India', 
                    'Mozambique', np.nan, 'Egypt', 'China', 'Colombia', 'South Sudan', 
                    'Nicaragua', 'Ireland', 'Iraq', 'Ukraine', 'Russian Federation',
                    'India', 'Algeria', 'United States', 'Afghanistan', 'Lebanon', 
                    np.nan, 'Mali', np.nan, 'Pakistan', 'Mali', 'Rwanda', 'Algeria', 
                    'Palestine', 'Palestine', 'Philippines', 'Sri Lanka', 'Syria', 
                    'Spain', 'Central African Republic', np.nan, 'Congo DRC', 
                    'Ethiopia', 'Philippines', np.nan, np.nan, 'Nepal', 'Ethiopia', 
                    'South Africa', 'India', 'Philippines', 'Afghanistan', 'Algeria', 
                    'Nicaragua', 'Sudan', 'Guatemala', 'Peru', 'Libya', 'Algeria', 
                    'Sierra Leone', 'Sudan', 'Colombia', 'Palestine', 'Rwanda', 'Iran', 
                    'India', np.nan, 'Congo DRC', 'Yemen', np.nan, 
                    'Egypt', 'India', 'Bangladesh', 'Afghanistan', 'Nicaragua', 
                    'Syria', 'Central African Republic', 'Libya', 'Russian Federation', 'India', 
                    np.nan, 'India', np.nan, 'Colombia', 'Philippines', 'China', np.nan,
                    'South Sudan', 'Pakistan', 'India', 'Northern Ireland', 'Ethiopia', 
                    'Cambodia', np.nan, 'South Africa', np.nan, 'Congo DRC', 
                    'Colombia', 'Indonesia', 'Iran', 'Palestine', 'Colombia', 'Palestine', 
                    'Central African Republic', 'Egypt', 'Syria', 'Yemen', np.nan, 'Syria', 
                    'Palestine', 'Nigeria', 'Lebanon', 'Syria', 'Guatemala', 'Palestine', 
                    np.nan, 'Mozambique', 'India', 'Tanzania', 'Sudan', 'India', np.nan,
                    'Pakistan', 'Russian Federation', np.nan, 'India', 'Ethiopia', 'Palestine', 'Central African Republic',
                    'Ethiopia', 'Pakistan', 'Nicaragua', 'Myanmar', 'Djibouti', 'Pakistan', 'Pakistan', 
                    'Senegal', 'Egypt', 'Russian Federation', 'Uganda', 'India', np.nan, 'Congo DRC',
                    'Algeria', 'South Sudan', 'South Sudan', np.nan, 'India', 'Argentina', 'Algeria', 
                    'Syria', 'Pakistan', 'Colombia', 'Northern Ireland', 'Liberia', 'Russian Federation', 
                    'India', 'Serbia', 'Congo DRC', 'Colombia', 
                    'Lebanon', np.nan, np.nan, 'Palestine', np.nan, 'Syria', 'Iraq', 'Iraq', 'Nigeria', 
                    'Pakistan', 'Iran', 'Turkey', 'Mali', np.nan, 'Ethiopia', np.nan, 'Rwanda', 
                    'India', 'Myanmar', 'Palestine', 'Algeria', np.nan, 'Egypt', np.nan, 
                    'Myanmar', 'Libya', np.nan, 'India', 'Mali', np.nan, 'Zimbabwe', np.nan, 
                    'Pakistan', 'Pakistan', 'Indonesia', 'Uganda', 'Syria', 'India', 'Mali', 
                    'Yemen', np.nan, 'Iraq', np.nan, 'El Salvador', 'Indonesia', 'Turkey',
                    'Myanmar', 'Libya', 'Iraq', 'Iraq', np.nan, 'Egypt', 'India',
                    'Algeria', 'Guatemala', 'Mali', 'Yemen']

# Add country of origin column
top_groups['Country'] = manual_locations

# Drop the nan countries
top_groups = top_groups.dropna(subset=['Country'])
top_groups = top_groups.rename(columns={'gname':'Group', 'nkill': 'Fatalities'})

GTD pickle file found, loading...


# 

#### Country location

To find the countries location on the map, we scrape the [World Countries Centroids](https://github.com/gavinr/world-countries-centroids) dataset. These locations denote the longitude and latitude of each country's centroid point. 


In [4]:
# Get country locations csv from url 
locations_url = 'https://cdn.jsdelivr.net/gh/gavinr/world-countries-centroids@v1/dist/countries.csv'
locations = pd.read_csv(locations_url, keep_default_na=False, na_values=['_'])
locations.head()

,longitude,latitude,COUNTRY,ISO,COUNTRYAFF,AFF_ISO
0,-170.700732,-14.305712,American Samoa,AS,United States,US
1,166.638003,19.302046,United States Minor Outlying Islands,UM,United States,US
2,-159.787689,-21.222613,Cook Islands,CK,New Zealand,NZ
3,-149.400417,-17.674684,French Polynesia,PF,France,FR
4,-169.868781,-19.052309,Niue,NU,New Zealand,NZ


In [5]:
# Print countries that have a missing location
missing_countries = [country for country in manual_locations if country not in locations['COUNTRY'].values]
print(set(missing_countries))

{nan, 'Northern Ireland', 'Palestine'}


As special cases, Palestine and Northern Ireland are missing a location from the dataset. We manually add them to the `countries.csv` file.

In [6]:
# Add row to locations for missing countries
locations = locations.append({'longitude': 54.607577, 'latitude': -6.693145, 'COUNTRY': 'Northern Ireland', 'ISO': 'GB', 'COUNTRYAFF': 'United Kingdom', 'AFF_ISO': 'GBR'}, ignore_index=True)
locations = locations.append({'longitude': 31.947351, 'latitude': 35.227163, 'COUNTRY': 'Palestine', 'ISO': 'PSE', 'COUNTRYAFF': 'Palestine', 'AFF_ISO': 'PSE'}, ignore_index=True)

#### Merging terrorist groups and their locations

Now that we have country locations, we are going to merge the terrorist groups' country of origin to each country location. We will then have a longitude and latitude for each terrorist group. 

In [7]:
# Add the longitude and latitude of each country by joining on Country name
locations = locations.rename(columns={'COUNTRY': 'Country'})
top_groups = top_groups.merge(locations[['Country', 'latitude', 'longitude']], on='Country', how='left')

# Save top_groups to csv file
top_groups = top_groups.rename(columns={'Group' : 'Terrorist group', 'latitude' : 'Group latitude', 'longitude' : 'Group longitude'})
top_groups.to_csv(os.path.join(MAP_DIR, 'top_groups.csv'), index=False)
top_groups

,Terrorist group,Fatalities,Country,Group latitude,Group longitude
0,Taliban,57650.0,Afghanistan,34.134026,66.592161
1,Islamic State of Iraq and the Levant (ISIL),43688.0,Iraq,33.105076,43.832529
2,Boko Haram,26339.0,Nigeria,9.610294,8.147715
3,Al-Shabaab,12740.0,Somalia,6.524535,45.400379
4,Shining Path (SL),11628.0,Peru,-8.522718,-74.114162
...,...,...,...,...,...
195,Khalistan Commando Force,118.0,India,23.586301,81.173004
196,Al-Mua'qi'oon Biddam Brigade (Those who Sign w...,117.0,Algeria,28.350970,2.655846
197,Guatemalan National Revolutionary Unity (URNG),115.0,Guatemala,15.820879,-90.312193
198,Ansar al-Dine (Mali),114.0,Mali,17.168146,-4.346400


#### Merging group locations with target locations

We now merge the above groups locations to have source and target coordinates in order to draw arcs from a terrorist group's country of origin to the location of its targets. 



In [8]:
# Add group location to GTD by joining on group name
GTD = GTD.merge(top_groups[['Terrorist group', 'Group latitude', 'Group longitude']], on='Terrorist group', how='left')

# Remove all attacks without group location
GTD = GTD.dropna(subset=['Group latitude', 'Group longitude'])
print('Number of attacks with identified group location:', len(GTD))
 
# Save to csv in group_attacks.csv
GTD.to_csv(os.path.join(MAP_DIR, 'group_attacks.csv'), index=False)
GTD



Number of attacks with identified group location: 89648


,datetime,City,Country,Region,Latitude,Longitude,Fatalities,Attack type,Terrorist group,Target type,Weapon used,Group latitude,Group longitude
38,1970-01-31 12:00:00,Bamban,Philippines,Southeast Asia,15.675051,120.331618,0.0,NaN,New People's Army (NPA),Military,NaN,15.586542,121.822089
58,1970-02-10 12:00:00,Munich,West Germany (FRG),Western Europe,48.139126,11.580186,1.0,Armed Assault,Popular Front for the Liberation of Palestine ...,Airports & Aircraft,Firearms,35.227163,31.947351
61,1970-02-13 12:00:00,Munich,West Germany (FRG),Western Europe,48.139126,11.580186,7.0,Facility/Infrastructure Attack,Popular Front for the Liberation of Palestine ...,Private Citizens & Property,Incendiary,35.227163,31.947351
78,1970-02-21 12:00:00,Frankfurt,West Germany (FRG),Western Europe,50.117970,8.644191,0.0,Bombing/Explosion,"Popular Front for the Liberation of Palestine,...",Airports & Aircraft,Explosives,35.227163,31.947351
79,1970-02-21 12:00:00,Zurich,Switzerland,Western Europe,47.368650,8.539182,47.0,Bombing/Explosion,"Popular Front for the Liberation of Palestine,...",Airports & Aircraft,Explosives,35.227163,31.947351
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209931,2021-06-30 12:00:00,Fortul,Colombia,South America,6.792500,-71.775833,0.0,Armed Assault,National Liberation Army of Colombia (ELN),Military,Explosives,4.187754,-72.644507
209932,2021-06-28 12:00:00,Alindao,Central African Republic,Sub-Saharan Africa,5.040041,21.210248,7.0,Armed Assault,Union for Peace in Central Africa (UPC),Government (Diplomatic),Firearms,6.331390,20.520743
209933,2021-06-30 12:00:00,Hassan Shallal,Iraq,Middle East & North Africa,35.048322,44.273285,NaN,Armed Assault,Islamic State of Iraq and the Levant (ISIL),Police,Firearms,33.105076,43.832529
209936,2021-06-30 12:00:00,NaN,Nigeria,Sub-Saharan Africa,11.572869,13.014035,0.0,Hostage Taking (Kidnapping),Boko Haram,Religious Figures/Institutions,Firearms,9.610294,8.147715
